<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


En este laboratorio, realizarás web scraping para recopilar registros históricos de lanzamientos de Falcon 9 de una página de Wikipedia titulada `List of Falcon 9 and Falcon Heavy launches` (Lista de lanzamientos de Falcon 9 y Falcon Heavy).

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [ ]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


En primer lugar, vamos a realizar un método HTTP GET para solicitar la página HTML Falcon9 Launch, como respuesta HTTP.

In [ ]:
# use requests.get() method with the provided static_url
response = requests.get(static_url)
# assign the response to a object
soup = BeautifulSoup(response.text, 'html.parser')

Crear un objeto `BeautifulSoup` a partir de la `response` HTML


In [ ]:
# Utilice BeautifulSoup() para crear un objeto BeautifulSoup a partir de un contenido de texto de respuesta

Imprime el título de la página para verificar si el objeto `BeautifulSoup` fue creado correctamente

In [ ]:
# Use soup.title attribute
# Obtener el título de la página
title = soup.title

# Imprimir el título
print(title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extraer todos los nombres de columnas/variables de la cabecera de la tabla HTML


A continuación, queremos recoger todos los nombres de columna relevantes de la cabecera de la tabla HTML

Intentemos encontrar primero todas las tablas en la página wiki. Si necesitas refrescar la memoria sobre `BeautifulSoup`, consulta el enlace de referencia externa al final de este laboratorio.

In [ ]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')


A partir de la tercera tabla es nuestra tabla de destino contiene los registros de lanzamiento real.

In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]


Debería poder ver los nombres de las columnas incrustados en los elementos de cabecera de la tabla `<th>` de la siguiente manera:

```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


A continuación, sólo tenemos que iterar a través de los elementos `<th>` y aplicar la función `extract_column_from_header()` para extraer el nombre de la columna uno a uno

In [ ]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [ ]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


Crearemos un diccionario vacío con las claves de los nombres de columna extraídos en la tarea anterior. Posteriormente, este diccionario se convertirá en un dataframe de Pandas

In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

A continuación, sólo tenemos que rellenar `launch_dict` con los registros de lanzamiento extraídos de las filas de la tabla.

Normalmente, las tablas HTML de las páginas Wiki suelen contener anotaciones inesperadas y otros tipos de ruidos, como enlaces de referencia `B0004.1[8]`, valores que faltan `N/A [e]`, formato incoherente, etc.

Para simplificar el proceso de análisis, hemos proporcionado un fragmento de código incompleto a continuación para ayudarle a rellenar el `launch_dict`. Por favor, complete el siguiente fragmento de código con TODOs o puede optar por escribir su propia lógica para analizar todas las tablas de lanzamiento:

In [ ]:
extracted_row = 0
#Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])

            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)

            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)

            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)

            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)

            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)

            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)


F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.17B10038
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 FT[
F9 v1.1[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT♺[
F9 FT[
F9 FT[
F9 FT[
F9 FTB1029.2195
F9 FT[
F9 FT[
F9 B4[
F9 FT[
F9 B4[
F9 B4[
F9 FTB1031.2220
F9 B4[
F9 FTB1035.2227
F9 FTB1036.2227
F9 B4[
F9 FTB1032.2245
F9 FTB1038.2268
F9 B4[
F9 B4B1041.2268
F9 B4B1039.2292
F9 B4[
F9 B5311B1046.1268
F9 B4B1043.2322
F9 B4B1040.2268
F9 B4B1045.2336
F9 B5
F9 B5349B1048[
F9 B5B1046.2354
F9 B5[
F9 B5B1048.2364
F9 B5B1047.2268
F9 B5B1046.3268
F9 B5[
F9 B5[
F9 B5B1049.2397
F9 B5B1048.3399
F9 B5[]413
F9 B5[
F9 B5B1049.3434
F9 B5B1051.2420
F9 B5B1056.2465
F9 B5B1047.3472
F9 B5
F9 B5[
F9 B5B1056.3482
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5
F9 B5
F9 B5B1058.2544
F9 B5
F9 B5B1049.6544
F9 B5
F9 B5B1060.2563
F9 B5B1058.3565
F9 B5B1051.6568


AttributeError: 'NoneType' object has no attribute 'string'

In [ ]:
extracted_row = 0

# Extract each table matching the specified attributes
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table rows
    for rows in table.find_all("tr"):
        # Check if the first table heading is a launch number
        if rows.th:
            flight_number = rows.th.string.strip() if rows.th.string else None
            flag = flight_number and flight_number.isdigit()
        else:
            flag = False

        # Get table elements (data cells)
        row = rows.find_all('td')

        # If it's a launch row (identified by flight number)
        if flag:
            extracted_row += 1

            # Populate the launch_dict with extracted data (handle potential missing elements)
            launch_dict['Flight No.'].append(flight_number)  # Append flight number (TODO: Append the flight_number into launch_dict with key `Flight No.`)

            datetime_list = date_time(row[0]) if len(row) > 0 else None  # Assuming you have a `date_time` function
            launch_dict['Date'].append(datetime_list[0].strip(',') if datetime_list else None)  # Append date (TODO: Append the date into launch_dict with key `Date`)
            launch_dict['Time'].append(datetime_list[1] if datetime_list else None)  # Append time (TODO: Append the time into launch_dict with key `Time`)

            bv = booster_version(row[1]) if len(row) > 1 else None  # Assuming you have a `booster_version` function
            launch_dict['Version Booster'].append(bv or row[1].a.string if len(row) > 1 and row[1].a else None)  # Append booster version (TODO: Append the bv into launch_dict with key `Version Booster`)
            print(bv)  # This line can be removed if not needed for further processing

            launch_site = row[2].a.string if len(row) > 2 and row[2].a else None  # Append launch site (TODO: Append the bv into launch_dict with key `Launch Site`)
            launch_dict['Launch site'].append(launch_site)

            payload = row[3].a.string if len(row) > 3 and row[3].a else None  # Append payload (TODO: Append the payload into launch_dict with key `Payload`)
            launch_dict['Payload'].append(payload)

            payload_mass = get_mass(row[4]) if len(row) > 4 else None  # Assuming you have a `get_mass` function
            launch_dict['Payload mass'].append(payload_mass)  # Append payload mass (TODO: Append the payload_mass into launch_dict with key `Payload mass`)

            orbit = row[5].a.string if len(row) > 5 and row[5].a else None  # Append orbit (TODO: Append the orbit into launch_dict with key `Orbit`)
            launch_dict['Orbit'].append(orbit)

            customer = row[6].a.string if len(row) > 6 and row[6].a else None  # Append customer (TODO: Append the customer into launch_dict with key `Customer`)
            launch_dict['Customer'].append(customer)

            launch_outcome = list(row[7].strings)[0] if len(row) > 7 else None  # Append launch outcome (TODO: Append the launch_outcome into launch_dict with key `Launch outcome`)
            launch_dict['Launch outcome'].append(launch_outcome)

            booster_landing = landing_status(row[8]) if len(row) > 8 else None  # Assuming you have a `landing_status` function
            launch_dict['Booster landing'].append(booster_landing)  # Append booster landing (TODO: Append the launch_outcome into launch_dict with key `Booster landing`)

# Now you have a populated launch_dict containing launch data!

F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.17B10038





F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 v1.1[
F9 FT[
F9 v1.1[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT[
F9 FT♺[
F9 FT[
F9 FT[
F9 FT[
F9 FTB1029.2195
F9 FT[
F9 FT[
F9 B4[
F9 FT[
F9 B4[
F9 B4[
F9 FTB1031.2220
F9 B4[
F9 FTB1035.2227
F9 FTB1036.2227
F9 B4[
F9 FTB1032.2245
F9 FTB1038.2268
F9 B4[
F9 B4B1041.2268
F9 B4B1039.2292
F9 B4[
F9 B5311B1046.1268
F9 B4B1043.2322
F9 B4B1040.2268
F9 B4B1045.2336
F9 B5
F9 B5349B1048[
F9 B5B1046.2354
F9 B5[
F9 B5B1048.2364
F9 B5B1047.2268
F9 B5B1046.3268
F9 B5[
F9 B5[
F9 B5B1049.2397
F9 B5B1048.3399
F9 B5[]413
F9 B5[
F9 B5B1049.3434
F9 B5B1051.2420
F9 B5B1056.2465
F9 B5B1047.3472
F9 B5
F9 B5[
F9 B5B1056.3482
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5
F9 B5
F9 B5B1058.2544
F9 B5
F9 B5B1049.6544
F9 B5
F9 B5B1060.2563
F9 B5B1058.3565
F9 B5B1051.6568
F9 B5
F9 B5
F9 B5[
F9 B5
F9 B5 ♺[
F

Una vez introducidos los valores de los registros de lanzamiento en `launch_dict`, puede crear un marco de datos a partir de ellos.

In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

Ahora podemos exportarlo a un <b>CSV</b> para la siguiente sección, pero para que las respuestas sean consistentes y en caso de que tengas dificultades para terminar este laboratorio.

Los siguientes laboratorios utilizarán un conjunto de datos proporcionado para que cada laboratorio sea independiente.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [ ]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
